# Пример работы с моделями открытых репозиториев:

## - Hugginface:

1. **Запуск моделей**:
    - загрузить биилиотеки для работы: requirements.txt
    - пройти на страницу моделей Hugginface: https://huggingface.co/models
    - выбрать теги для работы. наши теги по задачам:
        - Speech to text (STT): Audio и там automatic-speech-recognition, в "Filter by name" в меру воображения про русский язык (например, просто пишем  - ru )
        - Text Sentiment Analize (TSA): Natural Language Processing и там text-classification, а  в "Filter by name" в меру воображения про русский язык (например, просто пишем  - ru ) 
        - Speech to Emotion recognition (SER): Audio и там audio-classification, в "Filter by name" в меру воображения про русский язык (например, просто пишем  - ru ) - замечание : тут много моделей не про речь - читаем описание моделей и корректируем выбор.
    - Собираем список имен моделей:
        - открыть модель из списка оставшихся рабочих образцов со страницы 
        - запустить модель с примером из датасета (небольшое тестовое множество)
        - записать выходы и проверить адекватность записываемого 
    - Отбросить модели, которые решают не "нашу" задачу (не всегда прозрачно описан выход модели), и модели, которые не заработала (так тоже бывает)
 
 2. **Тест**:
    - Не верим результатам из карты модели - проверяем, но и себе не верим (проверяем)
    - Запускаем "Уцелевший список рабочих моделей" и записываем результаты в один параллельно заполняемый Дата фрейм
    - собрать таблицу результатов работы модели (дописать выходы модели к таблице с датасетом)
 
 3. **Анализ**:
    - провести очистку и интерпретицию результатов (модели не обязаны писать ответы как нам нужно, они пищут их как у них записано)
    - тестим на разных данных (датасеты, тоже имеют свою разметку)
    - пользуемся одной метрикой!!!!
 
 4. **Выводы**       
        

Пример для юболее продвинутого варианта использования моделей-  с доучиванием https://github.com/huggingface/notebooks/blob/main/examples/audio_classification.ipynb

## Audio classification

In [2]:

import pandas as pd
import numpy as np
import time

import warnings
warnings.simplefilter('ignore')


## инсталировать нужные пакеты

In [3]:
# # для установки torch (похоже tensorflow если используем его)
# !pip install torch torchvision
# # общий вариант
# !pip install transformers 
# # вариант с только с цпу
# # for torch
# !pip install 'transformers[torch]'
# # for tensorflow
# !pip install 'transformers[tf-cpu]'

# # другие варианты https://huggingface.co/docs/transformers/installation

In [4]:
# пакет от Hugginface

In [5]:
from transformers import pipeline


In [6]:
# серилизация объектов

In [7]:
import pickle

## Датасеты

  - **RESD** : 7 классов
  - https://huggingface.co/datasets/Aniemore/resd_annotated


In [8]:
path_resd_train = '../dataset/data_RESD.pickle'


with open(path_resd_train, 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    data = pickle.load(f)

In [9]:
data[0]

{'name': '32_happiness_enthusiasm_h_120',
 'path': 'happiness_enthusiasm_32/32_happiness_enthusiasm_h_120.wav',
 'emotion': 'happiness',
 'speech': {'path': '32_happiness_enthusiasm_h_120.wav',
  'array': array([-0.00018311, -0.00061035, -0.00076294, ...,  0.00085449,
          0.00048828,  0.00030518]),
  'sampling_rate': 16000}}

In [10]:
df_resd_train = pd.DataFrame(data)
df_resd_train.head()

,name,path,emotion,speech
0,32_happiness_enthusiasm_h_120,happiness_enthusiasm_32/32_happiness_enthusias...,happiness,"{'path': '32_happiness_enthusiasm_h_120.wav', ..."
1,36_disgust_happiness_d_130,disgust_happiness_36/36_disgust_happiness_d_13...,disgust,"{'path': '36_disgust_happiness_d_130.wav', 'ar..."
2,34_anger_fear_a_060,anger_fear_34/34_anger_fear_a_060.wav,anger,"{'path': '34_anger_fear_a_060.wav', 'array': [..."
3,25_anger_disgust_a_010,anger_disgust_25/25_anger_disgust_a_010.wav,anger,"{'path': '25_anger_disgust_a_010.wav', 'array'..."
4,17_neutral_disgust_d_092,neutral_disgust_17/17_neutral_disgust_d_092.wav,disgust,"{'path': '17_neutral_disgust_d_092.wav', 'arra..."


In [11]:
# from https://www.kaggle.com/datasets/ar4ikov/resd-dataset?resource=download

In [12]:
path_resd_test  = '../dataset/RESD_csv/test.csv'

In [13]:
df_resd = pd.read_csv(path_resd_test )
df_resd.head()

,name,path,emotion,text
0,27_neutral_fear_n_100,neutral_fear_27/27_neutral_fear_n_100.wav,neutral,"Вам дадут целый минимальный оклад, но при этом..."
1,08_sadness_anger a_010,08_sadness_anger/08_sadness_anger a_010.wav,anger,Сколько можно звонить?
2,26_enthusiasm_happiness_e_120,enthusiasm_happiness_26/26_enthusiasm_happines...,enthusiasm,А как долго тебе нужно это всё узнавать?
3,42_anger_fear_a_190,anger_fear_42/42_anger_fear_a_190.wav,anger,Ну а мне в 5 часов вставать на работу!
4,04_fear_enthusiasm f_090,04_fear_enthusiasm/04_fear_enthusiasm f_090.wav,fear,"Честно, я не подскажу, ну как и обычно, любым ..."


- сбалансирован

## Дополнение: Наш предполагаемый пайплайн обработки

оценка эмоций по звуку речи

In [14]:


import torch
from aniemore.recognizers.voice import VoiceRecognizer
from aniemore.models import HuggingFaceModel

model_w = HuggingFaceModel.Voice.WavLM
device = 'cuda' if torch.cuda.is_available() else 'cpu'
vr = VoiceRecognizer(model=model_w, device=device)

n = 0
wav_name = df_resd_train.name.iloc[n]
out_f =  '../dataset/RESD_train/' + str(n) + wav_name
vr.recognize(out_f, return_single_label=True)

Some weights of the model checkpoint at aniemore/wavlm-emotion-russian-resd were not used when initializing WavLMForSequenceClassification: ['wavlm.encoder.pos_conv_embed.conv.weight_v', 'wavlm.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing WavLMForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing WavLMForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of WavLMForSequenceClassification were not initialized from the model checkpoint at aniemore/wavlm-emotion-russian-resd and are newly initialized: ['wavlm.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wavlm.encoder.pos_conv_embed.conv.param

'happiness'

перевод речи в текст

In [15]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import Audio, load_dataset

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model_t = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="russian", task="transcribe")

wav_name = df_resd_train.name.iloc[n]
speech = df_resd_train.speech.iloc[n]
#
#  тут чтение и создание структуры {'array':[  numpy массив из аудио ], "sampling_rate":16000}
#

input_features = processor(speech["array"], sampling_rate=speech["sampling_rate"], return_tensors="pt").input_features

# generate token ids
predicted_ids = model_t.generate(input_features, forced_decoder_ids=forced_decoder_ids)
# decode token ids to text
transcription = processor.batch_decode(predicted_ids)

transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
transcription

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[' Конечно, скажу, обязательно. Ой, сейчас, ну скажу.']

Оценка эмоций по тексту

In [16]:
import torch
from aniemore.recognizers.text import TextRecognizer
from aniemore.models import HuggingFaceModel

model_e = HuggingFaceModel.Text.Bert_Tiny2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tr = TextRecognizer(model=model_e, device=device)

tr.recognize(transcription[0], return_single_label=True)

'happiness'

### Весь поток целиком

In [17]:
rez = [] 
for i in range(20):#df_resd_train.shape[0]):
    try:
        t1 =  time.time()
        wav_name = df_resd_train.name.iloc[i]
        speech = df_resd_train.speech.iloc[i]
        samples = speech['array']
        fs = speech['sampling_rate']
        out_f =  '../dataset/RESD_train/' + str(i) + wav_name

        s_em = vr.recognize(out_f, return_single_label=True)
        # SER
        input_features = processor(speech["array"], sampling_rate=speech["sampling_rate"], return_tensors="pt").input_features

        # STT
        predicted_ids = model_t.generate(input_features, forced_decoder_ids=forced_decoder_ids)
        # 
        
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
        
        #SA
        t_em = tr.recognize(transcription[0], return_single_label=True)

        # label_pred = model(processor(speech))

        label_true = df_resd_train.emotion.iloc[i]
        t1 =  time.time() - t1 
        rez.append([i, wav_name, label_true, s_em, t_em, t1, transcription[0]]) 
        # break
        print([i, wav_name, label_true, s_em, t_em, t1, transcription[0]])
    except:
        pass

[0, '32_happiness_enthusiasm_h_120', 'happiness', 'happiness', 'happiness', 1.2302322387695312, ' Конечно, скажу, обязательно. Ой, сейчас, ну скажу.']
[1, '36_disgust_happiness_d_130', 'disgust', 'disgust', 'neutral', 0.8124287128448486, ' Вы еще профессию решили поменять.']
[2, '34_anger_fear_a_060', 'anger', 'anger', 'enthusiasm', 0.8759293556213379, ' Ты знаешь, чем это для тебя закончится?']
[7, '32_happiness_enthusiasm_e_060', 'enthusiasm', 'enthusiasm', 'happiness', 1.5873656272888184, ' Ну хорошо, если например друзья, то что может быть латарейный билет и может им тоже повезет.']
[9, '33_sadness_disgust_s_080', 'sadness', 'sadness', 'enthusiasm', 0.878122091293335, ' Вжи одно то, что ты пришла, то уже праздник.']
[11, '32_happiness_enthusiasm_e_050', 'enthusiasm', 'enthusiasm', 'neutral', 1.1607391834259033, ' Так, ну и что ж ты купишь? Платили в сумму, а чеку может быть кошелёга для это.']
[15, '03_disgust_neutral n_050', 'neutral', 'neutral', 'anger', 1.036041498184204, ' Фино

In [18]:
df_aniemore_resd = pd.DataFrame(rez, columns=['N','file_name','label_true','label_audio','label_text', 't', 'text'])
df_aniemore_resd.to_csv('aniem_resd.csv')